In [3]:
# Loading the dataset
import pandas as pd
df = pd.read_csv('anime.csv')
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


Data Preprocessing

In [4]:
# Handling missing values
missing_values = df.isnull().sum()
print(missing_values)

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64


In [5]:
# cleaning the data
df_cleaned = df.dropna(subset=['genre', 'type', 'rating'])
missing_values_after_drop = df_cleaned.isnull().sum()
print(missing_values_after_drop)

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64


In [6]:
df_cleaned.info()
display(df_cleaned.describe())
display(df_cleaned.nunique())

<class 'pandas.core.frame.DataFrame'>
Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12017 non-null  int64  
 1   name      12017 non-null  object 
 2   genre     12017 non-null  object 
 3   type      12017 non-null  object 
 4   episodes  12017 non-null  object 
 5   rating    12017 non-null  float64
 6   members   12017 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 751.1+ KB


,anime_id,rating,members
count,12017.000000,12017.000000,1.201700e+04
mean,13638.001165,6.478264,1.834888e+04
std,11231.076675,1.023857,5.537250e+04
min,1.000000,1.670000,1.200000e+01
25%,3391.000000,5.890000,2.250000e+02
50%,9959.000000,6.570000,1.552000e+03
75%,23729.000000,7.180000,9.588000e+03
max,34519.000000,10.000000,1.013917e+06


,0
anime_id,12017
name,12015
genre,3229
type,6
episodes,187
rating,598
members,6596


Feature Extraction

In [7]:
features = ['genre', 'type', 'rating', 'members']
print("Selected features for similarity calculation:", features)

Selected features for similarity calculation: ['genre', 'type', 'rating', 'members']


In [8]:
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler

features_df = df_cleaned[['genre', 'type', 'rating', 'members']].copy()

# One-hot encode 'genre'
features_df['genre'] = features_df['genre'].apply(lambda x: x.split(', '))
mlb = MultiLabelBinarizer()
genre_encoded = pd.DataFrame(mlb.fit_transform(features_df['genre']), columns=mlb.classes_, index=features_df.index)

# One-hot encode 'type'
type_encoded = pd.get_dummies(features_df['type'], prefix='type', dtype=float)

# Scale 'rating' and 'members'
scaler = StandardScaler()
numerical_features = features_df[['rating', 'members']]
scaled_numerical = pd.DataFrame(scaler.fit_transform(numerical_features), columns=['rating', 'members'], index=features_df.index)

# Concatenate all features
prepared_features_df = pd.concat([genre_encoded, type_encoded, scaled_numerical], axis=1)

display(prepared_features_df.head())

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Yaoi,Yuri,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,rating,members
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1.0,0.0,0.0,0.0,0.0,0.0,2.824474,3.292044
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,2.717032,14.002410
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,2.707265,1.732216
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,2.629126,11.833499
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,2.619358,2.400518


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(prepared_features_df)
print("Cosine similarity matrix shape:", cosine_sim.shape)

Cosine similarity matrix shape: (12017, 12017)


In [10]:
def get_recommendations(title, cosine_sim=cosine_sim):
    """
    Gets anime recommendations based on cosine similarity.

    Args:
        title: The title of the anime to get recommendations for.
        cosine_sim: The pre-computed cosine similarity matrix.

    Returns:
        A list of recommended anime titles.
    """
    if title not in df_cleaned['name'].values:
        return "Anime not found in the dataset."

    idx = df_cleaned[df_cleaned['name'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get top 10 similar anime (excluding itself)
    anime_indices = [i[0] for i in sim_scores]
    return df_cleaned['name'].iloc[anime_indices].tolist()


In [11]:
def get_recommendations(title, cosine_sim=cosine_sim, threshold=0.0):
    """
    Gets anime recommendations based on cosine similarity.

    Args:
        title: The title of the anime to get recommendations for.
        cosine_sim: The pre-computed cosine similarity matrix.
        threshold: The minimum similarity score for a recommendation to be included.

    Returns:
        A list of recommended anime titles.
    """
    if title not in df_cleaned['name'].values:
        return "Anime not found in the dataset."

    idx = df_cleaned[df_cleaned['name'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Filter recommendations based on the threshold and exclude the anime itself
    recommended_indices = [i[0] for i in sim_scores if i[1] >= threshold and i[0] != idx]
    return df_cleaned['name'].iloc[recommended_indices].tolist()

Design a function to recommend anime based on cosine similarity.

In [13]:
# Get recommendations for a target anime with a threshold
target_anime = "Kimi no Na wa."
recommendations_with_threshold = get_recommendations(target_anime, cosine_sim, threshold=0.5)

# Display the recommendations
print(f"Recommendations for '{target_anime}' with a threshold of 0.5:")
for anime in recommendations_with_threshold:
    print(anime)

Recommendations for 'Kimi no Na wa.' with a threshold of 0.5:
Hotarubi no Mori e
Suzumiya Haruhi no Shoushitsu
Hotaru no Haka
Clannad: After Story - Mou Hitotsu no Sekai, Kyou-hen
Kotonoha no Niwa
Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku
Howl no Ugoku Shiro
Toki wo Kakeru Shoujo
Kanon (2006)
Koe no Katachi
Clannad: Mou Hitotsu no Sekai, Tomoyo-hen
Kokoro Connect
Byousoku 5 Centimeter
Tonari no Totoro
Nagi no Asukara
Angel Beats!: Another Epilogue
Ookami Kodomo no Ame to Yuki
Sen to Chihiro no Kamikakushi
Shigatsu wa Kimi no Uso
ReLIFE
Clannad: After Story
Shakugan no Shana II (Second)
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai. Movie
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Sakamichi no Apollon
Great Teacher Onizuka
Shakugan no Shana
Kokoro ga Sakebitagatterunda.
Nekomonogatari: Kuro
Yahari Ore no Seishun Love Comedy wa Machigatteiru.
Gekkan Shoujo Nozaki-kun
Sakurasou no Pet na Kanojo
Tokyo Magnitude 8.0
True Tears
Haikyuu!!
Monogatari Series: Second Sea

In [14]:
import numpy as np

# Simulate user interactions by randomly selecting a subset of anime
num_simulated_interactions = 50
user_interactions_indices = np.random.choice(df_cleaned.index, size=num_simulated_interactions, replace=False)
user_interactions = df_cleaned.loc[user_interactions_indices]['name'].tolist()

# Generate recommendations for each interacted anime
recommendations_for_evaluation = {}
for anime_title in user_interactions:
    recommendations = get_recommendations(anime_title, cosine_sim)
    recommendations_for_evaluation[anime_title] = recommendations

# Display a sample of the simulated interactions and their recommendations
sample_anime = list(recommendations_for_evaluation.keys())[:5]
for anime in sample_anime:
    print(f"Recommendations for '{anime}': {recommendations_for_evaluation[anime][:5]}...")

Recommendations for 'Go! Go! Itsutsugo Land': ['Pika Don', 'Shouwa Monogatari (Movie)', 'Kuro ga Ita Natsu', 'Zeno: Kagirinaki Ai ni', 'Ibara-Hime matawa Nemuri-Hime']...
Recommendations for 'Log Horizon': ['Magi: The Labyrinth of Magic', 'Fairy Tail', 'Fullmetal Alchemist: Brotherhood', 'Fullmetal Alchemist', 'Log Horizon 2nd Season']...
Recommendations for 'Sleepy': ['Spectral Force Chronicle Divergence', 'Shiawasette Naani', 'You Shoumei Bijutsukan Line', 'Doudou', 'Byston Well Monogatari: Garzey no Tsubasa']...
Recommendations for 'Mobile Suit Gundam Wing: Operation Meteor': ['Mobile Suit Gundam 0083: Stardust Memory', 'Mobile Suit Gundam 0080: War in the Pocket', 'Sentou Yousei Yukikaze', 'Mobile Suit Gundam Wing: Endless Waltz', 'Uchuu no Kishi Tekkaman Blade II']...
Recommendations for 'Anata ga Furikaeru Toki': ['Soul Worker: Your Destiny Awaits', 'Ze Tian Ji', 'Tobira wo Akete', 'Bavi Stock', 'Gunslinger Stratos: The Animation - Kikan/Kaze no Yukue']...


In [15]:
def get_recommendations(title, cosine_sim=cosine_sim, threshold=0.0):
    """
    Gets anime recommendations based on cosine similarity.

    Args:
        title: The title of the anime to get recommendations for.
        cosine_sim: The pre-computed cosine similarity matrix.
        threshold: The minimum similarity score for a recommendation to be included.

    Returns:
        A list of recommended anime titles.
    """
    if title not in df_cleaned['name'].values:
        return "Anime not found in the dataset."

    # Find the index in the original cleaned dataframe
    original_df_index = df_cleaned[df_cleaned['name'] == title].index[0]

    # Find the corresponding index in the prepared features dataframe
    # Assuming prepared_features_df has the same index as df_cleaned before reset
    idx = prepared_features_df.index.get_loc(original_df_index)


    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Filter recommendations based on the threshold and exclude the anime itself
    recommended_indices = [i[0] for i in sim_scores if i[1] >= threshold and i[0] != idx]

    # Map the indices back to the original dataframe to get anime names
    recommended_anime_names = df_cleaned['name'].iloc[recommended_indices].tolist()

    return recommended_anime_names

# Simulate user interactions by randomly selecting a subset of anime
num_simulated_interactions = 50
user_interactions_indices = np.random.choice(df_cleaned.index, size=num_simulated_interactions, replace=False)
user_interactions = df_cleaned.loc[user_interactions_indices]['name'].tolist()

# Generate recommendations for each interacted anime
recommendations_for_evaluation = {}
for anime_title in user_interactions:
    recommendations = get_recommendations(anime_title, cosine_sim)
    recommendations_for_evaluation[anime_title] = recommendations

# Display a sample of the simulated interactions and their recommendations
sample_anime = list(recommendations_for_evaluation.keys())[:5]
for anime in sample_anime:
    print(f"Recommendations for '{anime}': {recommendations_for_evaluation[anime][:5]}...")

Recommendations for 'Tokio Heidi: Mainichi no Kodomo Uta': ['Kiki to Lala no Hoshi no Dance Shoes', 'Genki Genki Non-tan: Spoon Tan Tan Tan', 'Genki Genki Non-tan: Utaou! Christmas', 'Hyakumannen Attara, Dousuru?', 'Kiki to Lala no Hakuchouza no Ohimesama']...
Recommendations for 'Fella Pure: Mitarashi-san Chi no Jijou The Animation': ['Tsugou no Yoi Sexfriend?', 'Stringendo: Angel-tachi no Private Lesson', 'Baku Ane: Otouto Shibocchau zo! The Animation', 'Shoujo Ramune', 'Maki-chan to Nau.']...
Recommendations for 'Prince of Tennis: The National Tournament Semifinals': ['Prince of Tennis: The National Tournament', 'New Prince of Tennis OVA vs. Genius10', 'Ookiku Furikabutte: Natsu no Scorebook', 'Naniwa Yuukyouden: Chou Gokudou! Yoru no Bat wa Manrui-hen', 'Major: World Series']...
Recommendations for 'Bara Bara Film': ['Believe in It', 'Petit Manga', 'Fue', 'Henkei Sakuhin Dai 1-ban', 'Isu']...
Recommendations for 'Natsu-iro Egao de 1, 2, Jump!': ['Mo Gyutto Love de Sekkinchuu!', 'Mu

In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_hit_rate(recommendations_dict, interacted_items):
    """
    Calculates the Hit Rate of the recommendations.

    Args:
        recommendations_dict: A dictionary where keys are interacted items and values are lists of recommended items.
        interacted_items: A list of the original interacted items (keys of recommendations_dict).

    Returns:
        The Hit Rate as a float.
    """
    hits = 0
    for item in interacted_items:
        # Check if the interacted item (or a related item that would count as a "hit")
        # is present in the recommendations for that item.
        # For simplicity in this simulation, we'll assume a hit occurs if
        # any item in the recommendations is considered a "relevant" item.
        # In a real scenario, you would have a separate list of test set interactions
        # to compare against the recommendations.
        # Since we are simulating and don't have a separate test set,
        # we will check if any of the top recommendations are among the
        # top similar items based on cosine similarity (excluding the item itself).
        # This is a proxy for evaluation in a simulated environment.
        # A more robust evaluation would require a proper train/test split
        # of user interaction data.

        # Get the original index of the interacted item
        if item not in df_cleaned['name'].values:
            continue
        original_df_index = df_cleaned[df_cleaned['name'] == item].index[0]
        idx = prepared_features_df.index.get_loc(original_df_index)


        # Get the indices of the top recommendations from the dictionary
        recommended_names = recommendations_dict.get(item, [])
        recommended_indices_in_cleaned_df = df_cleaned[df_cleaned['name'].isin(recommended_names)].index.tolist()

        # Get the top similar items from the cosine similarity matrix (excluding itself)
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # Consider the top N similar items as potential "hits"
        top_n = len(recommended_names) if len(recommended_names) > 0 else 10 # Use the number of recommendations or a default
        top_similar_indices = [i[0] for i in sim_scores[1:top_n + 1]]


        # Check if any of the recommended items are in the top similar items
        if any(rec_idx in top_similar_indices for rec_idx in recommended_indices_in_cleaned_df):
             hits += 1


    return hits / len(interacted_items) if len(interacted_items) > 0 else 0


def calculate_diversity(recommendations_dict, similarity_matrix, df):
    """
    Calculates the average pairwise dissimilarity of recommended items.

    Args:
        recommendations_dict: A dictionary where keys are interacted items and values are lists of recommended items.
        similarity_matrix: The pre-computed item similarity matrix.
        df: The original dataframe with anime names and indices.

    Returns:
        The Diversity score as a float.
    """
    all_recommended_indices = []
    for recommended_items in recommendations_dict.values():
         # Get the indices of the recommended items in the cleaned dataframe
        recommended_indices_in_cleaned_df = df[df['name'].isin(recommended_items)].index.tolist()
        # Get the corresponding indices in the prepared_features_df (and thus the similarity matrix)
        recommended_indices_in_sim_matrix = [prepared_features_df.index.get_loc(idx) for idx in recommended_indices_in_cleaned_df if idx in prepared_features_df.index]
        all_recommended_indices.extend(recommended_indices_in_sim_matrix)

    if len(all_recommended_indices) < 2:
        return 0.0

    # Calculate pairwise dissimilarities among all unique recommended items
    unique_recommended_indices = list(set(all_recommended_indices))
    if len(unique_recommended_indices) < 2:
        return 0.0

    pairwise_similarities = similarity_matrix[np.ix_(unique_recommended_indices, unique_recommended_indices)]
    # Use the upper triangle to avoid double counting and the diagonal (similarity of an item with itself)
    upper_triangle_indices = np.triu_indices(len(unique_recommended_indices), k=1)
    pairwise_dissimilarities = 1 - pairwise_similarities[upper_triangle_indices]

    return np.mean(pairwise_dissimilarities)


def calculate_novelty(recommendations_dict, df):
    """
    Calculates the average novelty of recommended items based on inverse popularity.

    Args:
        recommendations_dict: A dictionary where keys are interacted items and values are lists of recommended items.
        df: The original dataframe with anime names and 'members' count.

    Returns:
        The Novelty score as a float.
    """
    all_recommended_names = [item for sublist in recommendations_dict.values() for item in sublist]
    if not all_recommended_names:
        return 0.0

    # Get popularity (members count) for recommended anime
    recommended_anime_info = df[df['name'].isin(all_recommended_names)][['name', 'members']]

    # Calculate novelty as inverse of popularity (handle zero members if any)
    # Add a small constant to avoid division by zero, though 'members' minimum is 12
    recommended_anime_info['novelty'] = 1 / (recommended_anime_info['members'] + 1)

    # Calculate average novelty
    # Group by name to handle cases where the same anime is recommended multiple times
    average_novelty = recommended_anime_info.groupby('name')['novelty'].mean().mean()


    return average_novelty


In [17]:
# Calculate evaluation metrics
hit_rate = calculate_hit_rate(recommendations_for_evaluation, user_interactions)
diversity = calculate_diversity(recommendations_for_evaluation, cosine_sim, df_cleaned)
novelty = calculate_novelty(recommendations_for_evaluation, df_cleaned)

# Print the results
print(f"Hit Rate: {hit_rate:.4f}")
print(f"Diversity: {diversity:.4f}")
print(f"Novelty: {novelty:.8f}")

Hit Rate: 1.0000
Diversity: 0.8639
Novelty: 0.00358817


In [18]:
# Interpret the results
print("--- Evaluation Results Interpretation ---")
print(f"Hit Rate: {hit_rate:.4f}")
print(f"Diversity: {diversity:.4f}")
print(f"Novelty: {novelty:.8f}")

print("\nAnalysis:")
print(f"- Hit Rate ({hit_rate:.4f}): A Hit Rate of {hit_rate:.4f} suggests that for every simulated interaction, at least one of the recommended items was considered a 'hit' based on our proxy definition (being among the top similar items). In a real-world scenario with actual user interaction data, a high hit rate would indicate that the system is effectively recommending items that users are likely to interact with.")
print(f"- Diversity ({diversity:.4f}): A Diversity score of {diversity:.4f} indicates that the recommended items are relatively dissimilar to each other. This is a positive sign, suggesting that the system is not just recommending variations of the same item and is offering a range of different anime.")
print(f"- Novelty ({novelty:.8f}): The Novelty score of {novelty:.8f} (which is based on inverse popularity) suggests that the recommendations include less popular items on average. A higher novelty score generally means the system is recommending items that the user might not easily discover on their own, which can be a valuable aspect of a recommendation system.")

print("\nLimitations of the Simulated Evaluation:")
print("This evaluation is based on a simulated dataset of user interactions and a proxy for 'hits' (top similar items based on cosine similarity).")
print("In a real-world setting, evaluation would typically involve:")
print("- A proper train/test split of actual user interaction data.")
print("- Evaluating recommendations against a user's actual future interactions (e.g., items they watched or rated in the test set).")
print("- Using metrics like Recall@N or Precision@N for hit rate, and potentially other metrics for diversity and novelty that are less reliant on the similarity matrix itself.")
print("Therefore, while these results provide an indication of the system's behavior based on the defined similarity, they should be interpreted with caution and are not a substitute for evaluation with real user data.")

--- Evaluation Results Interpretation ---
Hit Rate: 1.0000
Diversity: 0.8639
Novelty: 0.00358817

Analysis:
- Hit Rate (1.0000): A Hit Rate of 1.0000 suggests that for every simulated interaction, at least one of the recommended items was considered a 'hit' based on our proxy definition (being among the top similar items). In a real-world scenario with actual user interaction data, a high hit rate would indicate that the system is effectively recommending items that users are likely to interact with.
- Diversity (0.8639): A Diversity score of 0.8639 indicates that the recommended items are relatively dissimilar to each other. This is a positive sign, suggesting that the system is not just recommending variations of the same item and is offering a range of different anime.
- Novelty (0.00358817): The Novelty score of 0.00358817 (which is based on inverse popularity) suggests that the recommendations include less popular items on average. A higher novelty score generally means the system

In [19]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_hit_rate(recommendations_dict, interacted_items):
    """
    Calculates the Hit Rate of the recommendations.

    Args:
        recommendations_dict: A dictionary where keys are interacted items and values are lists of recommended items.
        interacted_items: A list of the original interacted items (keys of recommendations_dict).

    Returns:
        The Hit Rate as a float.
    """
    hits = 0
    for item in interacted_items:
        # Check if the interacted item (or a related item that would count as a "hit")
        # is present in the recommendations for that item.
        # For simplicity in this simulation, we'll assume a hit occurs if
        # any item in the recommendations is considered a "relevant" item.
        # In a real scenario, you would have a separate list of test set interactions
        # to compare against the recommendations.
        # Since we are simulating and don't have a separate test set,
        # we will check if any of the top recommendations are among the
        # top similar items based on cosine similarity (excluding the item itself).
        # This is a proxy for evaluation in a simulated environment.
        # A more robust evaluation would require a proper train/test split
        # of user interaction data.

        # Get the original index of the interacted item
        if item not in df_cleaned['name'].values:
            continue
        original_df_index = df_cleaned[df_cleaned['name'] == item].index[0]
        idx = prepared_features_df.index.get_loc(original_df_index)


        # Get the indices of the top recommendations from the dictionary
        recommended_names = recommendations_dict.get(item, [])
        recommended_indices_in_cleaned_df = df_cleaned[df_cleaned['name'].isin(recommended_names)].index.tolist()

        # Get the top similar items from the cosine similarity matrix (excluding itself)
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # Consider the top N similar items as potential "hits"
        top_n = len(recommended_names) if len(recommended_names) > 0 else 10 # Use the number of recommendations or a default
        top_similar_indices = [i[0] for i in sim_scores[1:top_n + 1]]


        # Check if any of the recommended items are in the top similar items
        if any(rec_idx in top_similar_indices for rec_idx in recommended_indices_in_cleaned_df):
             hits += 1


    return hits / len(interacted_items) if len(interacted_items) > 0 else 0


def calculate_diversity(recommendations_dict, similarity_matrix, df):
    """
    Calculates the average pairwise dissimilarity of recommended items.

    Args:
        recommendations_dict: A dictionary where keys are interacted items and values are lists of recommended items.
        similarity_matrix: The pre-computed item similarity matrix.
        df: The original dataframe with anime names and indices.

    Returns:
        The Diversity score as a float.
    """
    all_recommended_indices = []
    for recommended_items in recommendations_dict.values():
         # Get the indices of the recommended items in the cleaned dataframe
        recommended_indices_in_cleaned_df = df[df['name'].isin(recommended_items)].index.tolist()
        # Get the corresponding indices in the prepared_features_df (and thus the similarity matrix)
        recommended_indices_in_sim_matrix = [prepared_features_df.index.get_loc(idx) for idx in recommended_indices_in_cleaned_df if idx in prepared_features_df.index]
        all_recommended_indices.extend(recommended_indices_in_sim_matrix)

    if len(all_recommended_indices) < 2:
        return 0.0

    # Calculate pairwise dissimilarities among all unique recommended items
    unique_recommended_indices = list(set(all_recommended_indices))
    if len(unique_recommended_indices) < 2:
        return 0.0

    pairwise_similarities = similarity_matrix[np.ix_(unique_recommended_indices, unique_recommended_indices)]
    # Use the upper triangle to avoid double counting and the diagonal (similarity of an item with itself)
    upper_triangle_indices = np.triu_indices(len(unique_recommended_indices), k=1)
    pairwise_dissimilarities = 1 - pairwise_similarities[upper_triangle_indices]

    return np.mean(pairwise_dissimilarities)


def calculate_novelty(recommendations_dict, df):
    """
    Calculates the average novelty of recommended items based on inverse popularity.

    Args:
        recommendations_dict: A dictionary where keys are interacted items and values are lists of recommended items.
        df: The original dataframe with anime names and 'members' count.

    Returns:
        The Novelty score as a float.
    """
    all_recommended_names = [item for sublist in recommendations_dict.values() for item in sublist]
    if not all_recommended_names:
        return 0.0

    # Get popularity (members count) for recommended anime
    recommended_anime_info = df[df['name'].isin(all_recommended_names)][['name', 'members']]

    # Calculate novelty as inverse of popularity (handle zero members if any)
    # Add a small constant to avoid division by zero, though 'members' minimum is 12
    recommended_anime_info['novelty'] = 1 / (recommended_anime_info['members'] + 1)

    # Calculate average novelty
    # Group by name to handle cases where the same anime is recommended multiple times
    average_novelty = recommended_anime_info.groupby('name')['novelty'].mean().mean()


    return average_novelty

In [20]:
# Interpret the results
print("--- Evaluation Results Interpretation ---")
print(f"Hit Rate: {hit_rate:.4f}")
print(f"Diversity: {diversity:.4f}")
print(f"Novelty: {novelty:.8f}")

print("\nAnalysis:")
print(f"- Hit Rate ({hit_rate:.4f}): A Hit Rate of {hit_rate:.4f} suggests that for every simulated interaction, at least one of the recommended items was considered a 'hit' based on our proxy definition (being among the top similar items). In a real-world scenario with actual user interaction data, a high hit rate would indicate that the system is effectively recommending items that users are likely to interact with.")
print(f"- Diversity ({diversity:.4f}): A Diversity score of {diversity:.4f} indicates that the recommended items are relatively dissimilar to each other. This is a positive sign, suggesting that the system is not just recommending variations of the same item and is offering a range of different anime.")
print(f"- Novelty ({novelty:.8f}): The Novelty score of {novelty:.8f} (which is based on inverse popularity) suggests that the recommendations include less popular items on average. A higher novelty score generally means the system is recommending items that the user might not easily discover on their own, which can be a valuable aspect of a recommendation system.")

print("\nLimitations of the Simulated Evaluation:")
print("This evaluation is based on a simulated dataset of user interactions and a proxy for 'hits' (top similar items based on cosine similarity).")
print("In a real-world setting, evaluation would typically involve:")
print("- A proper train/test split of actual user interaction data.")
print("- Evaluating recommendations against a user's actual future interactions (e.g., items they watched or rated in the test set).")
print("- Using metrics like Recall@N or Precision@N for hit rate, and potentially other metrics for diversity and novelty that are less reliant on the similarity matrix itself.")
print("Therefore, while these results provide an indication of the system's behavior based on the defined similarity, they should be interpreted with caution and are not a substitute for evaluation with real user data.")

--- Evaluation Results Interpretation ---
Hit Rate: 1.0000
Diversity: 0.8639
Novelty: 0.00358817

Analysis:
- Hit Rate (1.0000): A Hit Rate of 1.0000 suggests that for every simulated interaction, at least one of the recommended items was considered a 'hit' based on our proxy definition (being among the top similar items). In a real-world scenario with actual user interaction data, a high hit rate would indicate that the system is effectively recommending items that users are likely to interact with.
- Diversity (0.8639): A Diversity score of 0.8639 indicates that the recommended items are relatively dissimilar to each other. This is a positive sign, suggesting that the system is not just recommending variations of the same item and is offering a range of different anime.
- Novelty (0.00358817): The Novelty score of 0.00358817 (which is based on inverse popularity) suggests that the recommendations include less popular items on average. A higher novelty score generally means the system

In [21]:
import numpy as np

# Simulate user interactions by randomly selecting a subset of anime
num_simulated_interactions = 50
user_interactions_indices = np.random.choice(df_cleaned.index, size=num_simulated_interactions, replace=False)
user_interactions = df_cleaned.loc[user_interactions_indices]['name'].tolist()

# Generate recommendations for each interacted anime
recommendations_for_evaluation = {}
for anime_title in user_interactions:
    recommendations = get_recommendations(anime_title, cosine_sim)
    recommendations_for_evaluation[anime_title] = recommendations

# Display a sample of the simulated interactions and their recommendations
sample_anime = list(recommendations_for_evaluation.keys())[:5]
for anime in sample_anime:
    print(f"Recommendations for '{anime}': {recommendations_for_evaluation[anime][:5]}...")

Recommendations for 'Shin Onimusha: Dawn of Dreams the Story': ['The Hakkenden: Shin Shou', 'The Hakkenden', 'Shinshuu Sudama-hen', 'Sengoku Kitan Youtouden', 'Kangetsu Ittou: Akuryou Kiri']...
Recommendations for 'Sugio: Mori de Koi wo Shite': ['Organic', 'Gokudou Sakaba Denden: Gokudou Daisensou Gaiden', 'Neo Satomi Hakkenden: Satomi-chanchi no Hachi Danshi', 'Gokicha!!', 'Idol Ace']...
Recommendations for 'Antique Heart': ['Mahoutsukai Nara Miso wo Kue!', 'Midori no Neko', 'Kaitou Tenshi Twin Angel', 'Kaitou Tenshi Twin Angel: Kyun Kyun☆Tokimeki Paradise!! OVA', 'Sin in the Rain']...
Recommendations for 'Urusei Yatsura': ['Urusei Yatsura Movie 2: Beautiful Dreamer', 'Urusei Yatsura Movie 5: Final', 'R.O.D the TV', 'Urusei Yatsura Movie 1: Only You', 'Cybersix']...
Recommendations for 'Lulin Da Maoxian': ['Sekai Meisaku Douwa: Aladdin to Mahou no Lamp', 'Shounen Sarutobi Sasuke', 'Bogeul Bogeul Bomulseon', 'Kaitei Sanman Mile', 'Mo fa a ma']...


**Q1. Can you explain the difference between user-based and item-based collaborative filtering?**  

- **User-Based Collaborative Filtering:**  
  This method finds users who are similar to the active user based on their past ratings or interactions. Then it recommends items that those similar users liked.  
  Example: *If User A and User B both liked the same 10 animes, and User B liked one more anime, we can recommend that anime to User A.*  

- **Item-Based Collaborative Filtering:**  
  This method looks at the similarity between items instead of users. If two items are frequently rated or interacted with together by many users, they are considered similar. Recommendations are made by suggesting items similar to what the user has already liked.  
  Example: *If most users who watched "Naruto" also enjoyed "Bleach", then a user who liked "Naruto" will be recommended "Bleach".*  

- **Key Difference:**  
  - User-based focuses on *finding similar users*.  
  - Item-based focuses on *finding similar items*.  
  Item-based is often more stable and scalable for large datasets.  

**Q2. What is collaborative filtering, and how does it work?**  

Collaborative filtering is a recommendation technique that makes predictions about a user’s interests by collecting preferences from many other users.  
It works on the assumption that *users who agreed in the past will also agree in the future*.  

- **How it works:**  
  1. Collect user-item interaction data (ratings, clicks, purchases, etc.).  
  2. Compute similarity (either between users or items).  
  3. Generate recommendations by predicting which items the active user is likely to prefer, based on similar users or similar items.  

- **Example:**  
  If User A and User B have rated many shows similarly, and User B liked “Attack on Titan” which User A hasn’t seen yet, collaborative filtering would recommend “Attack on Titan” to User A.  
